# Movie Rating Prediction with Stacked Autoencoders in PyTorch

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
ratings = pd.read_csv('ratings_small.csv')#, engine = 'python')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
ratings.drop(["timestamp"], axis=1)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [6]:
movieMax = max(ratings["movieId"])
movieMax

163949

In [7]:
userMax = max(ratings["userId"])
userMax

671

In [8]:
user1 = ratings.loc[ratings["userId"] == 4]
user1

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346
...,...,...,...,...
346,4,3251,5.0,949918970
347,4,3255,4.0,949919738
348,4,3263,3.0,949919845
349,4,3265,5.0,949895732


In [9]:
movieMin = min(ratings["movieId"])
movieMin

1

In [10]:
id_movies = ratings.loc[ratings["userId"] == 2]["movieId"]
id_movies

20     10
21     17
22     39
23     47
24     50
     ... 
91    592
92    593
93    616
94    661
95    720
Name: movieId, Length: 76, dtype: int64

In [11]:
id_ratings = list(ratings.loc[ratings["userId"] == 2]["rating"])
id_ratings

[4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 3.0,
 3.0,
 4.0,
 3.0,
 5.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 5.0,
 1.0,
 3.0,
 3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 5.0,
 3.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 2.0,
 1.0,
 3.0,
 4.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 3.0,
 4.0,
 3.0,
 4.0,
 2.0,
 4.0,
 3.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 3.0,
 5.0,
 3.0,
 5.0,
 3.0,
 3.0,
 3.0,
 5.0,
 5.0,
 5.0,
 3.0,
 3.0,
 4.0,
 4.0]

In [12]:
def convert(data):
    new_data = []
    for user in range(1, max(ratings["userId"]+1)):
        id_movies = data.loc[ratings["userId"] == user]["movieId"]
        id_ratings = data.loc[ratings["userId"] == user]["rating"]
        userRatings = np.zeros(movieMax)
        userRatings[id_movies-1] = id_ratings
        new_data.append(userRatings)
    return(new_data)

In [13]:
ratings = convert(ratings)

In [14]:
ratings = np.asarray(ratings)
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [15]:
x_train ,x_test = train_test_split(ratings, test_size=0.2)

In [16]:
len(x_train[1])

163949

In [17]:
len(x_test)

135

In [18]:
x_train = torch.Tensor(x_train)
x_test= torch.Tensor(x_test)

In [19]:
class SAE(nn.Module):
    def __init__(self, nb_movies):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [20]:
sae = SAE(movieMax)

In [21]:
sae.to(device)

SAE(
  (fc1): Linear(in_features=163949, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=163949, bias=True)
  (activation): Sigmoid()
)

In [22]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [23]:
len(x_train)

536

In [24]:
x_train = Variable(x_train).unsqueeze(0)

In [25]:
y_train = x_train.clone()

In [26]:
y_train = y_train.cuda()
y_train.is_cuda

True

In [27]:
x_test = x_test.cuda()
x_test.is_cuda

True

In [28]:
x_train = x_train.cuda()
x_train.is_cuda

True

In [29]:
next(sae.parameters()).is_cuda 

True

In [30]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(len(x_train)):
        target = x_train[id_user]
        input1 = x_train[id_user]
        target.to(device)
        input1.to(device)
        if torch.sum(target.data > 0) > 0:
            output = sae(input1)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = movieMax/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += torch.sqrt((loss.data)*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.1592, device='cuda:0')
epoch: 2 loss: tensor(0.1588, device='cuda:0')
epoch: 3 loss: tensor(0.1586, device='cuda:0')
epoch: 4 loss: tensor(0.1586, device='cuda:0')
epoch: 5 loss: tensor(0.1586, device='cuda:0')
epoch: 6 loss: tensor(0.1586, device='cuda:0')
epoch: 7 loss: tensor(0.1586, device='cuda:0')
epoch: 8 loss: tensor(0.1586, device='cuda:0')
epoch: 9 loss: tensor(0.1586, device='cuda:0')
epoch: 10 loss: tensor(0.1586, device='cuda:0')
epoch: 11 loss: tensor(0.1586, device='cuda:0')
epoch: 12 loss: tensor(0.1586, device='cuda:0')
epoch: 13 loss: tensor(0.1586, device='cuda:0')
epoch: 14 loss: tensor(0.1586, device='cuda:0')
epoch: 15 loss: tensor(0.1586, device='cuda:0')
epoch: 16 loss: tensor(0.1586, device='cuda:0')
epoch: 17 loss: tensor(0.1586, device='cuda:0')
epoch: 18 loss: tensor(0.1586, device='cuda:0')
epoch: 19 loss: tensor(0.1586, device='cuda:0')
epoch: 20 loss: tensor(0.1585, device='cuda:0')
epoch: 21 loss: tensor(0.1585, device='cuda:0')
e

epoch: 171 loss: tensor(0.1582, device='cuda:0')
epoch: 172 loss: tensor(0.1582, device='cuda:0')
epoch: 173 loss: tensor(0.1582, device='cuda:0')
epoch: 174 loss: tensor(0.1582, device='cuda:0')
epoch: 175 loss: tensor(0.1582, device='cuda:0')
epoch: 176 loss: tensor(0.1582, device='cuda:0')
epoch: 177 loss: tensor(0.1582, device='cuda:0')
epoch: 178 loss: tensor(0.1582, device='cuda:0')
epoch: 179 loss: tensor(0.1582, device='cuda:0')
epoch: 180 loss: tensor(0.1582, device='cuda:0')
epoch: 181 loss: tensor(0.1582, device='cuda:0')
epoch: 182 loss: tensor(0.1582, device='cuda:0')
epoch: 183 loss: tensor(0.1582, device='cuda:0')
epoch: 184 loss: tensor(0.1582, device='cuda:0')
epoch: 185 loss: tensor(0.1582, device='cuda:0')
epoch: 186 loss: tensor(0.1582, device='cuda:0')
epoch: 187 loss: tensor(0.1581, device='cuda:0')
epoch: 188 loss: tensor(0.1582, device='cuda:0')
epoch: 189 loss: tensor(0.1581, device='cuda:0')
epoch: 190 loss: tensor(0.1582, device='cuda:0')
epoch: 191 loss: ten

In [31]:
#sae.to("cpu")